In [0]:
import joblib

trained_model=joblib.load('random_forest_model.pkl')

In [0]:
import mlflow
from mlflow.models import infer_signature
import numpy as np
from sklearn.model_selection import train_test_split


# Đọc dữ liệu và xử lý
path = '/mnt/gold/gold/'
df = spark.read.format('delta').load(path)

required_columns = [
    'is_injured', 'contributing_factor_vehicle_1_encoded', 'contributing_factor_vehicle_2_encoded',
    'contributing_factor_vehicle_3_encoded', 'contributing_factor_vehicle_4_encoded',
    'contributing_factor_vehicle_5_encoded', 'vehicle_type_code1_encoded', 
    'vehicle_type_code2_encoded', 'borough_encoded'
]
df_pd = df.select(required_columns).toPandas()

# Chia dữ liệu thành đặc trưng và nhãn
X = df_pd.drop(columns=['is_injured'])
y = df_pd['is_injured']

# Chia tập dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Bắt đầu ghi log trong MLflow
with mlflow.start_run(run_name="Prophet Model Run") as run:
    # Ghi signature của mô hình
    signature = infer_signature(X_train, y_train)

    # Đăng ký và ghi mô hình với MLflow
    reg_model_name = "random_forest_model"
    print("--")
    model_log=mlflow.sklearn.log_model(
        sk_model=trained_model,
        artifact_path="random_forest_model",
        serialization_format="cloudpickle",
        signature=signature,
        registered_model_name=reg_model_name,
    )

    # Lấy run_id để theo dõi sau này
    run_id = run.info.run_uuid
    print(f"Run ID: {run_id}")


/databricks/python/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


--


Registered model 'random_forest_model' already exists. Creating a new version of this model...
2024/12/01 05:53:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 3


Run ID: c4a61d47400c4b11b830de026f3ed814


Created version '3' of model 'random_forest_model'.
